In [ ]:
from torchsummary import summary
import torch.autograd as autograd
from random import random, weibullvariate
import opendatasets as od
import os                 # Dient zum lokalen Speichern des Datasets
import numpy as np
import torch as t
import torchvision
from torchvision import datasets
import torch.utils.data as DataLoader
import torchvision.datasets as ImageFolder
import torchvision.transforms as transforms  # Transformieren von Bildern
import matplotlib.pyplot as plt  # plotten von Grafen/ Bildern
import torch.nn as nn  # Neuronales Netz
import torch.optim as optim  # Optimierungs-Algorithmen
from torchvision.utils import save_image  # Speichern von Bildern
from matplotlib import image
from tkinter.tix import IMAGE
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
from torchvision.utils import save_image
from torch.autograd import Variable
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import torch
!pip install opendatasets
!pip install torchsummary

#import torch.nn.functional as F  # Loss
#!pip install opendatasets

IMAGE_SIZE = 28  # Größe der Bilder
BATCH_SIZE = 128  # Anzahl der Batches
WORKERS = 2  # Anzahl der Kerne beim Arbeiten auf der GPU
# Normalisierung mit 0.5 Mittelwert und Standardabweichung für alle drei Channels der Bilder
NORM = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)

data_dir = '../data/'
os.makedirs(data_dir, exist_ok=True)  # Anlegen eines Ordners für Bilder

# Erklärung zum Umgang mit Opendata und Kaggle - https://pypi.org/project/opendatasets/
# Datensatz:Anime-Faces werden von Kaggle geladen
# Hierfür wird der User-API-KEY benötigt
# APIKEY {"username":"kimmhl","key":"f585163b4ee30f0a5b44b1a902dc56e6"}
dataset_url = 'https://www.kaggle.com/splcher/animefacedataset'
# Images werden in './animefacedataset' gespeichert
od.download(dataset_url, data_dir)

"""
Ausgabe zu den Ordnern
"""
print(os.listdir(data_dir))  # zeigt Ordner an

# gibt 10 Bezeichnungen von Bildern aus
print(os.listdir(data_dir+'animefacedataset/images')[:10])


# Transformer
transform = transforms.Compose([
    # Resize der Images auf 64 der kürzesten Seite; Andere Seite wird
    transforms.Resize(IMAGE_SIZE),
    # skaliert, um das Seitenverhältnis des Bildes beizubehalten.
    # Zuschneiden auf die Mitte des Images, sodass ein quadratisches Bild mit 64 x 64 Pixeln entsteht
    transforms.CenterCrop(IMAGE_SIZE),
    # Umwandeln in einen Tensor (Bildern in numerische Werte umwandeln)
    transforms.ToTensor(),
    transforms.Normalize(*NORM)])          # Normalisierung Mean & Standardabweichung von 0.5 für alle Channels
# (Anzahl: 3 für farbige Bilder)
# Pixelwerte liegen damit zwischen (-1;1)

# Dataset
"""
ImageFolder() : Befehl erwartet, dass nach Images nach labeln organisiert sind (root/label/picture.png)
"""
org_dataset = torchvision.datasets.ImageFolder(
    root=data_dir, transform=transform)

# Dataloader
"""
Dataloader():
"""
org_loader = t.utils.data.DataLoader(org_dataset,              # Dataset (Images)
                                     # Es wird auf Batches trainiert, damit auf Basis eines Batch-Fehlers das NN angepasst wird
                                     batch_size=BATCH_SIZE,
                                     shuffle=True,
                                     num_workers=WORKERS)

# Nutzen der GPU wenn vorhanden, ansonsten CPU


def get_default_device():
    if t.cuda.is_available():     # Wenn cuda verfügbar dann:
        return t.device('cuda')   # Nutze Device = Cuda (=GPU)
    else:                         # Ansonsten
        return t.device('cpu')    # Nutze Device = CPU


# Anzeigen welches Device verfügbar ist
device = get_default_device()
print(device)

# Hilfsklasse zum Verschieben des Dataloaders "org_loader" auf das jeweilige Device


class DeviceDataLoader():

    # Initialisierung
    def __init__(self, dataloader, device):
        self.dataloader = dataloader
        self.device = device

    # Anzahl der Images pro Batch
    def __len__(self):
        return len(self.dataloader)

    # Erstellt einen Batch an Tensoren nach dem Verschieben auf das Device
    def __iter__(self):
        for batch in self.dataloader:
            yield tuple(tensor.to(self.device) for tensor in batch)


# Dataloader auf dem verfügbaren Device
dataloader = DeviceDataLoader(org_loader, device)


def get_noise(n_samples, noise_dim, device=device):
    '''
    Generate noise vectors from the random normal distribution with dimensions (n_samples, noise_dim),
    where
        n_samples: the number of samples to generate based on  batch_size
        noise_dim: the dimension of the noise vector
        device: device type can be cuda or cpu
    '''

    return torch.randn(n_samples, noise_dim, 1, 1, device=device)


class Generator(nn.Module):
    def __init__(self, no_of_channels=3, noise_dim=100, gen_dim=64):
      super(Generator, self).__init__()
      self.network = nn.Sequential(
          nn.ConvTranspose2d(noise_dim, gen_dim*4, 4, 1, 0, bias=False),
          nn.BatchNorm2d(gen_dim*4),
          nn.ReLU(True),

          nn.ConvTranspose2d(gen_dim*4, gen_dim*2, 3, 2, 1, bias=False),
          nn.BatchNorm2d(gen_dim*2),
          nn.ReLU(True),

          nn.ConvTranspose2d(gen_dim*2, gen_dim, 4, 2, 1, bias=False),
          nn.BatchNorm2d(gen_dim),
          nn.ReLU(True),

          nn.ConvTranspose2d(gen_dim, no_of_channels, 4, 2, 1, bias=False),
          nn.Tanh()
      )

    def forward(self, input):
      output = self.network(input)
      return output


class Discriminator(nn.Module):
    def __init__(self, no_of_channels=3, disc_dim=64):
        super(Discriminator, self).__init__()
        self.network = nn.Sequential(

            nn.Conv2d(no_of_channels, disc_dim, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(disc_dim, disc_dim * 2, 4, 2, 1, bias=False),
            nn.InstanceNorm2d(disc_dim * 2, affine=True),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(disc_dim * 2, disc_dim * 4, 3, 2, 1, bias=False),
            nn.InstanceNorm2d(disc_dim * 4, affine=True),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(disc_dim * 4, 1, 4, 1, 0, bias=False),

        )

    def forward(self, input):
        output = self.network(input)
        #return output.view(-1, 1).squeeze(1)
        return output


gen = Generator().to(device)
critic = Discriminator().to(device)

# You initialize the weights to the normal distribution
# with mean 0 and standard deviation 0.02


def weights_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.normal_(m.weight, mean=0.0, std=0.02)
    if isinstance(m, nn.BatchNorm2d):
        torch.nn.init.normal_(m.weight, mean=0.0, std=0.02)
        torch.nn.init.constant_(m.bias, val=0)


gen = gen.apply(weights_init)
critic = critic.apply(weights_init)

lr = 1e-4
criterion = nn.BCEWithLogitsLoss()
gen_opt = torch.optim.RMSprop(gen.parameters(), lr=lr)
critic_opt = torch.optim.RMSprop(critic.parameters(), lr=lr)


def gradient_penalty(critic, real_image, fake_image, device=device):
    batch_size, channel, height, width = real_image.shape
    #alpha is selected randomly between 0 and 1
    alpha = torch.rand(batch_size, 1, 1, 1).repeat(
        1, channel, height, width).to(device)
    # interpolated image=randomly weighted average between a real and fake image
    #interpolated image ← alpha *real image  + (1 − alpha) fake image
    interpolatted_image = (alpha*real_image) + (1-alpha) * fake_image

    # calculate the critic score on the interpolated image
    interpolated_score = critic(interpolatted_image)

    # take the gradient of the score wrt to the interpolated image
    gradient = torch.autograd.grad(inputs=interpolatted_image,
                                   outputs=interpolated_score,
                                   retain_graph=True,
                                   create_graph=True,
                                   grad_outputs=torch.ones_like(
                                       interpolated_score)
                                   )[0]
    gradient = gradient.view(gradient.shape[0], -1)
    gradient_norm = gradient.norm(2, dim=1)
    gradient_penalty = torch.mean((gradient_norm-1)**2)
    return gradient_penalty

# Hilfsfunktionen zur Normalisierng von Tensoren und grafischen Darstellung


def tensor_norm(img_tensors):
    # print (img_tensors)
    # print (img_tensors * NORM [1][0] + NORM [0][0])
    return img_tensors * NORM[1][0] + NORM[0][0]


def show_images(images, nmax=64):
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.set_xticks([])
    ax.set_yticks([])
    plt.title("Fake_Images")
    ax.imshow(make_grid(tensor_norm(images.detach()[:nmax]), nrow=8).permute(
        1, 2, 0).cpu())  # detach() : erstellt eine neue "Ansicht",
    # sodass diese Operationen nicht mehr verfolgt werden,orm
    # d. h. der Gradient wird nicht berechnet und der Untergraph
    # wird nicht aufgezeichnet > Speicher wird nicht verwendet


"""
Ordner anlegen für die vom Generator erstellten Images
"""

dir_gen_samples = '../data/outputs/'
#os.makedirs('../outputs/dir_gen_samples', exist_ok=True)
os.makedirs(dir_gen_samples, exist_ok=True)


def saves_gen_samples(idx, random_Tensor):
    # Randomisierter Tensor wird an den Generator übergeben
    fake_img = gen(random_Tensor)
    # Setzen von Bildbezeichnungen für die Fake_Images
    fake_img_name = "gen_img-{0:0=4d}.png".format(idx)
    # Tensor-Normalisierung; Speichern der Fake_Images im Ordner "Outputs/dir_gen_samples/"
    save_image(tensor_norm(fake_img), os.path.join(
        dir_gen_samples, fake_img_name), nrow=8)
    show_images(fake_img)  # Plotten der Fake_Images
    print("Gespeichert")


def display_images(image_tensor, num_images=25, size=(3, 28, 28)):

    flatten_image = image_tensor.detach().cpu().view(-1, *size)
    image_grid = make_grid(flatten_image[:num_images], nrow=5)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    plt.show()


# Training

n_epochs = 3
cur_step = 0
LAMBDA_GP = 10
display_step = 500
z_dim = 100
CRITIC_ITERATIONS = 5
G_losses = []
D_losses = []
start_idx = 1

for epoch in range(n_epochs):
    # Dataloader returns the batches
    for real_image, _ in tqdm(dataloader):
        cur_batch_size = real_image.shape[0]

        real_image = real_image.to(device)

        for _ in range(CRITIC_ITERATIONS):
            fake_noise = get_noise(cur_batch_size, z_dim, device=device)
            fake = gen(fake_noise)
            critic_fake_pred = critic(fake).reshape(-1)
            critic_real_pred = critic(real_image).reshape(-1)
            #Calculate gradient penalty on real and fake images generated by generator
            gp = gradient_penalty(critic, real_image, fake, device)
            critic_loss = -(torch.mean(critic_real_pred) -
                            torch.mean(critic_fake_pred)) + LAMBDA_GP * gp

            critic.zero_grad()
            #To make a backward pass and retain the intermediary results
            critic_loss.backward(retain_graph=True)
            critic_opt.step()

        # Train Generator: max E[critic(gen_fake)] <-> min -E[critic(gen_fake)]
        gen_fake = critic(fake).reshape(-1)
        gen_loss = -torch.mean(gen_fake)

        #print( "D_loss:", critic_loss, "G_Loss:", gen_loss)

        gen.zero_grad()
        gen_loss.backward()
        # Update optimizer
        gen_opt.step()

        ## Visualization code ##
        if cur_step % display_step == 0 and cur_step > 0:
            print(
                f"Step {cur_step}: Generator loss: {gen_loss}, critic loss: {critic_loss}")
            display_images(fake)
            display_images(real_image)
            gen_loss = 0
            critic_loss = 0
        cur_step += 1


In [ ]:
!pip install opendatasets
!pip install torchsummary

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from tkinter.tix import IMAGE
from matplotlib import image
#import torch.nn.functional as F  # Loss
from torchvision.utils import save_image  # Speichern von Bildern
import torch.optim as optim  # Optimierungs-Algorithmen
import torch.nn as nn  # Neuronales Netz
from torchvision.utils import make_grid
import matplotlib.pyplot as plt  # plotten von Grafen/ Bildern
import torchvision.transforms as transforms  # Transformieren von Bildern
import torchvision.datasets as ImageFolder
import torch.utils.data as DataLoader
from torchvision import datasets
import torchvision
import torch as t
import numpy as np
import os                 # Dient zum lokalen Speichern des Datasets
import opendatasets as od
from random import random, weibullvariate
from torch.autograd import Variable
import torch.autograd as autograd
from torchsummary import summary
#!pip install opendatasets

IMAGE_SIZE = 64  # Größe der Bilder
BATCH_SIZE = 128  # Anzahl der Batches
WORKERS = 2  # Anzahl der Kerne beim Arbeiten auf der GPU
# Normalisierung mit 0.5 Mittelwert und Standardabweichung für alle drei Channels der Bilder
NORM = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)

data_dir = './data/'
os.makedirs(data_dir, exist_ok=True)  # Anlegen eines Ordners für Bilder

# Erklärung zum Umgang mit Opendata und Kaggle - https://pypi.org/project/opendatasets/
# Datensatz:Anime-Faces werden von Kaggle geladen
# Hierfür wird der User-API-KEY benötigt
# APIKEY {"username":"kimmhl","key":"f585163b4ee30f0a5b44b1a902dc56e6"}
dataset_url = 'https://www.kaggle.com/splcher/animefacedataset'
# Images werden in './animefacedataset' gespeichert
od.download(dataset_url, data_dir)

"""
Ausgabe zu den Ordnern
"""
print(os.listdir(data_dir))  # zeigt Ordner an

# gibt 10 Bezeichnungen von Bildern aus
print(os.listdir(data_dir+'animefacedataset/images')[:10])


# Transformer
transform = transforms.Compose([
    # Resize der Images auf 64 der kürzesten Seite; Andere Seite wird
    transforms.Resize(IMAGE_SIZE),
    # skaliert, um das Seitenverhältnis des Bildes beizubehalten.
    # Zuschneiden auf die Mitte des Images, sodass ein quadratisches Bild mit 64 x 64 Pixeln entsteht
    transforms.CenterCrop(IMAGE_SIZE),
    # Umwandeln in einen Tensor (Bildern in numerische Werte umwandeln)
    transforms.ToTensor(),
    transforms.Normalize(*NORM)])          # Normalisierung Mean & Standardabweichung von 0.5 für alle Channels
# (Anzahl: 3 für farbige Bilder)
# Pixelwerte liegen damit zwischen (-1;1)

# Dataset
"""
ImageFolder() : Befehl erwartet, dass nach Images nach labeln organisiert sind (root/label/picture.png)
"""
org_dataset = torchvision.datasets.ImageFolder(root=data_dir, transform=transform)

# Dataloader
"""
Dataloader():
"""
org_loader = t.utils.data.DataLoader(org_dataset,              # Dataset (Images)
                                     # Es wird auf Batches trainiert, damit auf Basis eines Batch-Fehlers das NN angepasst wird
                                     batch_size=BATCH_SIZE,
                                     shuffle=True,
                                     num_workers=WORKERS)

# Nutzen der GPU wenn vorhanden, ansonsten CPU


def get_default_device():
    if t.cuda.is_available():     # Wenn cuda verfügbar dann:
        return t.device('cuda')   # Nutze Device = Cuda (=GPU)
    else:                         # Ansonsten
        return t.device('cpu')    # Nutze Device = CPU


# Anzeigen welches Device verfügbar ist
device = get_default_device()
print(device)

# Hilfsklasse zum Verschieben des Dataloaders "org_loader" auf das jeweilige Device


class DeviceDataLoader():

    # Initialisierung
    def __init__(self, dataloader, device):
        self.dataloader = dataloader
        self.device = device

    # Anzahl der Images pro Batch
    def __len__(self):
        return len(self.dataloader)

    # Erstellt einen Batch an Tensoren nach dem Verschieben auf das Device
    def __iter__(self):
        for batch in self.dataloader:
            yield tuple(tensor.to(self.device) for tensor in batch)


# Dataloader auf dem verfügbaren Device
dataloader = DeviceDataLoader(org_loader, device)

                      
def get_noise(n_samples, noise_dim, device=device):
    '''
    Generate noise vectors from the random normal distribution with dimensions (n_samples, noise_dim),
    where
        n_samples: the number of samples to generate based on  batch_size
        noise_dim: the dimension of the noise vector
        device: device type can be cuda or cpu
    '''
    
    return  torch.randn(n_samples,noise_dim, 1,1,device=device)

                      

class Generator(nn.Module):
    def __init__(self, no_of_channels=3, noise_dim=100, gen_dim=64):
      super(Generator, self).__init__()
      self.network = nn.Sequential(
          nn.ConvTranspose2d(noise_dim, gen_dim*8, 4, 1, 0, bias=False),
          nn.BatchNorm2d(gen_dim*8),
          nn.ReLU(True),
  
          nn.ConvTranspose2d(gen_dim*8, gen_dim*4, 4, 2, 1, bias=False),
          nn.BatchNorm2d(gen_dim*4),
          nn.ReLU(True),
  
          nn.ConvTranspose2d(gen_dim*4, gen_dim*2, 4, 2, 1, bias=False),
          nn.BatchNorm2d(gen_dim*2),
          nn.ReLU(True),
          
          nn.ConvTranspose2d(gen_dim*2, gen_dim, 4, 2, 1, bias=False),
          nn.BatchNorm2d(gen_dim),
          nn.ReLU(True),
  
          nn.ConvTranspose2d(gen_dim, no_of_channels, 4, 2, 1, bias=False),
          nn.Tanh()
      )
  
    def forward(self, input):
      output = self.network(input)
      return output

class Discriminator(nn.Module):
    def __init__(self, no_of_channels=3, disc_dim=64):
        super(Discriminator, self).__init__()
        self.network = nn.Sequential(
                
                nn.Conv2d(no_of_channels, disc_dim, 4, 2, 1, bias=False),
                nn.LeakyReLU(0.2, inplace=True),
                
                nn.Conv2d(disc_dim, disc_dim * 2, 4, 2, 1, bias=False),
                nn.InstanceNorm2d(disc_dim * 2, affine=True),
                nn.LeakyReLU(0.2, inplace=True),
                
                nn.Conv2d(disc_dim * 2, disc_dim * 4, 3, 2, 1, bias=False),
                nn.InstanceNorm2d(disc_dim * 4, affine=True),
                nn.LeakyReLU(0.2, inplace=True),
            
                nn.Conv2d(disc_dim * 4, disc_dim * 8, 3, 2, 1, bias=False),
                nn.InstanceNorm2d(disc_dim * 8, affine=True),
                nn.LeakyReLU(0.2, inplace=True),
                
                nn.Conv2d(disc_dim * 8, 1, 4, 1, 0, bias=False),
                
            )
    def forward(self, input):
        output = self.network(input)
        return output.view(-1, 1).squeeze(1)
        #return output
                      
gen = Generator().to(device)
critic =Discriminator().to(device)
                      
# You initialize the weights to the normal distribution                      
# with mean 0 and standard deviation 0.02
def weights_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.normal_(m.weight, mean=0.0, std=0.02)
    if isinstance(m, nn.BatchNorm2d):
        torch.nn.init.normal_(m.weight, mean=0.0, std=0.02)
        torch.nn.init.constant_(m.bias, val=0)
gen = gen.apply(weights_init)
critic = critic.apply(weights_init)
                      
lr = 1e-4
criterion = nn.BCEWithLogitsLoss()
gen_opt = torch.optim.RMSprop(gen.parameters(), lr=lr)
critic_opt = torch.optim.RMSprop(critic.parameters(), lr=lr)
                      
                      
                      
def gradient_penalty( critic, real_image, fake_image, device=device):
    batch_size, channel, height, width= real_image.shape
    #alpha is selected randomly between 0 and 1
    alpha= torch.rand(batch_size,1,1,1).repeat(1, channel, height, width).to(device)
    # interpolated image=randomly weighted average between a real and fake image
    #interpolated image ← alpha *real image  + (1 − alpha) fake image
    interpolatted_image=(alpha*real_image) + (1-alpha) * fake_image
    
    # calculate the critic score on the interpolated image
    interpolated_score= critic(interpolatted_image)
    
    # take the gradient of the score wrt to the interpolated image
    gradient= torch.autograd.grad(inputs=interpolatted_image,
                                  outputs=interpolated_score,
                                  retain_graph=True,
                                  create_graph=True,
                                  grad_outputs=torch.ones_like(interpolated_score)                          
                                 )[0]
    gradient= gradient.view(gradient.shape[0],-1)
    gradient_norm= gradient.norm(2,dim=1)
    gradient_penalty=torch.mean((gradient_norm-1)**2)
    return gradient_penalty

# Hilfsfunktionen zur Normalisierng von Tensoren und grafischen Darstellung
def tensor_norm(img_tensors):
    # print (img_tensors)
    # print (img_tensors * NORM [1][0] + NORM [0][0])
    return img_tensors * NORM[1][0] + NORM[0][0]

def show_images(images, nmax=64):
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.set_xticks([])
    ax.set_yticks([])
    plt.title("Fake_Images")
    ax.imshow(make_grid(tensor_norm(images.detach()[:nmax]), nrow=8).permute(
        1, 2, 0).cpu())  # detach() : erstellt eine neue "Ansicht",
    # sodass diese Operationen nicht mehr verfolgt werden,orm
    # d. h. der Gradient wird nicht berechnet und der Untergraph
    # wird nicht aufgezeichnet > Speicher wird nicht verwendet

"""
Ordner anlegen für die vom Generator erstellten Images
"""

dir_gen_samples = '../data/outputs/'
#os.makedirs('../outputs/dir_gen_samples', exist_ok=True)
os.makedirs(dir_gen_samples, exist_ok=True)    
    
def saves_gen_samples(idx, random_Tensor):
    # Randomisierter Tensor wird an den Generator übergeben
    fake_img = gen(random_Tensor)
    # Setzen von Bildbezeichnungen für die Fake_Images
    fake_img_name = "gen_img-{0:0=4d}.png".format(idx)
    # Tensor-Normalisierung; Speichern der Fake_Images im Ordner "Outputs/dir_gen_samples/"
    save_image(tensor_norm(fake_img), os.path.join(
        dir_gen_samples, fake_img_name), nrow=8)
    show_images(fake_img)  # Plotten der Fake_Images
    print("Gespeichert")


def display_images(image_tensor, num_images=25, size=(3, 64, 64)):
    
    flatten_image = image_tensor.detach().cpu().view(-1, *size)
    image_grid = make_grid(flatten_image[:num_images], nrow=5)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    plt.show()


# Training 
                      
n_epochs = 3
cur_step = 0
LAMBDA_GP = 10
display_step = 500
z_dim=100
CRITIC_ITERATIONS = 5
G_losses = []
D_losses = []
start_idx = 1

for epoch in range(n_epochs):
    # Dataloader returns the batches
    for real_image, _ in tqdm(dataloader):
        cur_batch_size = real_image.shape[0]
    
        real_image = real_image.to(device)
    
        for _ in range(CRITIC_ITERATIONS):
            fake_noise = get_noise(cur_batch_size, z_dim, device=device)
            fake = gen(fake_noise)
            critic_fake_pred = critic(fake).reshape(-1)
            critic_real_pred = critic(real_image).reshape(-1)
            #Calculate gradient penalty on real and fake images generated by generator
            gp=gradient_penalty(critic, real_image, fake, device)
            critic_loss = -(torch.mean(critic_real_pred) -torch.mean(critic_fake_pred)) + LAMBDA_GP *gp
            
            critic.zero_grad()
            #To make a backward pass and retain the intermediary results
            critic_loss.backward(retain_graph=True)
            critic_opt.step()
            
        # Train Generator: max E[critic(gen_fake)] <-> min -E[critic(gen_fake)]
        gen_fake= critic(fake).reshape(-1)
        gen_loss = -torch.mean(gen_fake)
        
        #print( "D_loss:", critic_loss, "G_Loss:", gen_loss)

        gen.zero_grad()
        gen_loss.backward()
        # Update optimizer
        gen_opt.step()
        
        ## Visualization code ##
        if cur_step % display_step == 0 and cur_step > 0:
            print(f"Step {cur_step}: Generator loss: {gen_loss}, critic loss: {critic_loss}")
            display_images(fake)
            display_images(real_image)
            gen_loss = 0
            critic_loss = 0
        cur_step += 1





In [41]:
from docx import *

document = Document('/Users/User/Downloads/AbwA_74.docx')
bolds=[]
italics=[]
for para in document.paragraphs:
    for run in para.runs:
        if run.italic :
            italics.append(run.text)
        if run.bold :
            bolds.append(run.text)

boltalic_Dict={'bold_phrases':bolds,
              'italic_phrases':italics}

print(boltalic_Dict)

{'bold_phrases': ['Antrag', 'Entscheidung'], 'italic_phrases': ['Antrag', 'Entscheidung']}
